In [1]:
import pathlib
from pathlib import Path
import pandas as pd
import xarray as xr
import numpy as np
from peak_resolver.process_chromeleon import process_chromeleon_file

In [45]:
def get_fanny_concentration(file: pathlib.PosixPath):
    n = file.name
    conc = {'1':0.05, '2':0.1, '3': 0.2, '4':0.3, '5': 0.4, '6':0.5}
    return conc[n[1]]

def read_fanny_calib(p: pathlib.PosixPath, tmin=0, tmax=30, tgrid=None,
                    acids=['Acetic', 'Propanoic', 'Butyric', 'Valeric', 'Caproic']):
    indices = acids[:]
    indices.append('t')
    # indices.append('t')
    
    
    files = p.glob('*.txt')
    acid = p.name.split(' ')[0].lower().capitalize()
    print(acid)
    data_list = []
    for file in files:
        data = process_chromeleon_file(file, tmin, tmax, tgrid = tgrid)
        
        for i in acids:
            data[i] = 0
            
        data[acid] = get_fanny_concentration(file)

        data = data.set_index(indices)
        data = data.to_xarray()
        data_list.append(data)
    data = xr.combine_by_coords(data_list)
    savepath = str(p / acid) + '.nc'
    data.to_netcdf(savepath)
    return files, data, acid

In [47]:
path = Path('/Users/ianbillinge/Documents/yiplab/projects/org_acids_iems/Fanny Raw IC Data/Calibration Curves/')
path = path.resolve()
acids = [i for i in path.iterdir() if i.is_dir()]
acids = tuple(acids)

files = []
data = []
names = []
# [print(i.name) for i in acids]
for acid in acids:
    # print(acid.name)

    f, d, name = read_fanny_calib(acid, tmin=6.5, tmax=9, 
                            tgrid = np.linspace(0.15, 19.85, 5997)
                           )
    files.append(f)
    data.append(d)
    names.append(name)
# data = xr.concat(data, names)
# data = xr.merge(data, compat='override') 
data = xr.combine_by_coords(data)

Acetic
Butyric
Propanoic
Valeric
Caproic


In [48]:
data

<xarray.Dataset>
Dimensions:    (Acetic: 7, t: 761, Propanoic: 7, Butyric: 7, Valeric: 7,
                Caproic: 7)
Coordinates:
  * Acetic     (Acetic) float64 0.0 0.05 0.1 0.2 0.3 0.4 0.5
  * t          (t) float64 6.501 6.504 6.507 6.511 ... 8.988 8.991 8.995 8.998
  * Propanoic  (Propanoic) float64 0.0 0.05 0.1 0.2 0.3 0.4 0.5
  * Butyric    (Butyric) float64 0.0 0.05 0.1 0.2 0.3 0.4 0.5
  * Valeric    (Valeric) float64 0.0 0.05 0.1 0.2 0.3 0.4 0.5
  * Caproic    (Caproic) float64 0.0 0.05 0.1 0.2 0.3 0.4 0.5
Data variables:
    s          (Acetic, Propanoic, Butyric, Valeric, Caproic, t) float64 nan ...
    ds         (Acetic, Propanoic, Butyric, Valeric, Caproic, t) float64 nan ...
    d2s        (Acetic, Propanoic, Butyric, Valeric, Caproic, t) float64 nan ...

In [49]:
fp = '/Users/ianbillinge/Documents/yiplab/projects/org_acids_iems/Fanny Raw IC Data/Calibration Curves'
fp += '/calib.nc'
data.to_netcdf(fp)
               